<a href="https://colab.research.google.com/github/EisaacJC/Ciencia-de-Datos-Personal/blob/master/Comparison_between_discretization_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <font size="5"><h3 align="center">Comparativa de desempeño de algoritmos de discretización en diferentes algortimos de clasificación</h3></font>

<div style="text-align: justify">Resumen: Se implementó el algoritmo de discrestización conocido como class-attribute interdependence maximization (CAIM) [1]. Con la finalidad de obtener una comparativa, se realizaron discretizaciones usando una transformación por k-medias y cuantil, disponibles en la librería de sklearn.
A partir de los conjuntos discretizados y adicionando los conjuntos originales se utilizaron tres algoritmos de clasificación: Naive Bayes, DecisionTreeClassifier y Support vector machine (SVM). </div>

<div style="text-align: justify">Los resultados de la clasificación muestran una mejora significativa al discretizar las bases de datos.
</div>

<font size="2.5">Integrantes:</font> \\
<font size="2.5">Diana Itzel Vazquez Santiago </font> \\
<font size="2.5">Emmanuel Isaac Juarez Caballero </font> \\
<font size="2.5">Jesús Eduardo Hermosilla Díaz </font> 

## Methodology

### Libraries

In [1]:
import urllib.request
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
import csv
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

### Webreader

Para el proceso de pre-procesamiento se extrajeron 10 bases de datos provenientes del [Machine Learning Repository]("https://archive.ics.uci.edu/ml/index.php"), los cuales se encuentran un repositorio de [github]("https://raw.githubusercontent.com/EisaacJC/datasets/main/").

Su lectura se realiza a partir de la función webreader y downloader.

In [2]:
#lectura de datos
def webreader(url):
    urls=pd.read_csv(url+"enlaces.txt", header=None)
    urls=urls.values.tolist()
    urlsp=[]
    urlsp2=[]
    for i in range(len(urls)):
        urlsp.append(urls[i][0])
        urlsp2.append(url+urls[i][0])
    urls=urlsp
    return urls,urlsp2

In [3]:
urls=webreader("https://raw.githubusercontent.com/EisaacJC/datasets/main/")

In [4]:
def downloader(urls):
    urls=urls
    for i in range(len(urls[0])):
        url=urls[1][i]
        urllib.request.urlretrieve(url, '/content/'+urls[0][i])
    return True    

In [5]:
downloader(urls)

True

### CAIM code

Para realizar la discretización CAIM se empleó el siguiente pseudo-código: \\

Dado un conjunto de $M$ ejemplos, $S$ clases y $F_i$ atributos continuos. \\
Para cada $F_i$ hacer: \\
- Paso 1. 
 - Encontrar el valor máximo ($d_n$), y mínimo ($d_0$) de $F_i$.
 - Formar un conjunto con todos los valores distintos de $F_i$ en orden ascendente, e inicializar todos los posibles intervalos límites $B$ con mínimo, máximo y todos los puntos medios de todos los pares adyacente en el conjunto.
 - Establecer el esquema de inicialización inicial como $D:\{[d_0,d_n]\}$, fijar $GlobalCAIM=0$. 
- Paso 2.
 - Inicializar $k=1$.
 - Agregar tentativamente los límites inferiores de $B$ que no estén en $D$ y calcular el valor $CAIM$ correspondiente.
 - Después de que todas las adiciones tentativas hayan sido probadas aceptar la que tiene el mayor valor de $CAIM$.
 - Si ($CAIM$ > $GlobalCAIM$ or $k< S$) actualizar $D$ con el límite aceptado en el paso anterior y fijar $GlobalCAIM=CAIM$, sino terminar.
 - Fijar $k=k+1$ y regresar al segundo punto del paso dos.

 Salida: El esquema $D$ discretizado.



In [6]:
def get_caim(scheme, data, target):
  B = []
  for p in scheme[1:-1]:
    B.append(np.where(data > p)[0][0])

  B.insert(0, 0)
  B.append(data.shape[0])
  n = len(B) - 1
  isum = 0
  for j in range(n):
    init = B[j]
    fin = B[j + 1]
    Mr = data[init:fin].shape[0]
    val, counts = np.unique(target[init:fin], return_counts=True)
    maxr = counts.max()
    isum = isum + (maxr / Mr) * maxr
  return isum/n

def transform(X, scheme):
  X_di = X.copy()
  for j in range(X.shape[1]):
    sh = scheme[j]
    sh[-1] = sh[-1] + 1
    xj = X[:, j]
    for i in range(len(sh) - 1):
      ind = np.where((xj >= sh[i]) & (xj < sh[i + 1]))[0]
      X_di[ind, j] = i
  return X_di

def fit(X, target):
  split_scheme = dict()
  min_splits = np.unique(target).shape[0]

  for j in range(X.shape[1]):
    xj = X[:, j]
    xj = xj[np.invert(np.isnan(xj))]
    new_index = xj.argsort()
    xj = xj[new_index]
    yj = target[new_index]
    allsplits = np.unique(xj)[1:-1].tolist()
    global_caim = -1
    mainscheme = [xj[0], xj[-1]]
    best_caim = 0
    k = 1

    while (k <= min_splits) or ((global_caim < best_caim) and (allsplits)):
      split_points = np.random.permutation(allsplits).tolist()
      best_scheme = None
      best_point = None
      best_caim = 0
      k = k + 1
      while split_points:
        scheme = mainscheme[:]
        B = split_points.pop()
        scheme.append(B)
        scheme.sort()
        c = get_caim(scheme, xj, yj)
        if c > best_caim:
          best_caim = c
          best_scheme = scheme
          best_point = B
      if (k <= min_splits) or (best_caim > global_caim):
        mainscheme = best_scheme
        global_caim = best_caim
        allsplits.remove(best_point)
    split_scheme[j] = mainscheme
  return split_scheme

In [7]:
def main_caim(x,y):
  return transform(x,fit(x,y))

### Naive Bayes classifier

<div style="text-align: justify">Uno de los clasificadores más conocidos, simples y eficientes es el clasificador Naive Bayes el cual es un modelo probabilístico de aprendizaje automático que se utiliza para tareas de clasificación de datos, su funcionamiento se basa en la probabilidad condicional y el teorema de Bayes.</div>

\begin{align}
P(C|F_1,...,F_n) = \frac{P(C)P(F_1,...,F_n|C)}{P(F_1,...,F_n)}
\end{align}

Se utilizó la implementación desarrollada para la tarea uno de la materia.

In [8]:
def training(data_train, target_train, trainingSize, numFeatures):
    casesByFeature = {}
    data = []
    for i in range(trainingSize):
        if not target_train[i] in casesByFeature.keys():
            casesByFeature[target_train[i]] = 1
        else:
            casesByFeature[target_train[i]] += 1
        data.append([data_train[i], target_train[i]])
    return casesByFeature, data

def classify(numFeatures, trainingSize, casesByFeature, data, test):
    solve = None
    max_arg = 0
    for y in casesByFeature.keys():
        prob = casesByFeature[y]/trainingSize
        for i in range(numFeatures):
            cases = [x for x in data if x[0][i] == test[i] and x[1] == y]
            n = len(cases)
            prob *= n/trainingSize
        if prob > max_arg:
            max_arg = prob
            solve = y
    return solve

def naive_ingenuo(instances, classes, percentage):
  data_train, data_test, target_train, target_test = train_test_split(instances, classes, test_size=percentage)
  trainingSize = len(data_train)
  numFeatures = len(data_train[0])
  testingSize = len(target_test)
  good = 0

  casesByFeature, data = training(data_train, target_train, trainingSize, numFeatures)
  for i in range(testingSize):
    predict = classify(numFeatures, trainingSize, casesByFeature, data, data_test[i])
    if target_test[i] == predict:
        good += 1
  
  dict_score = dict([('Instancias',testingSize), ('Bien clasificadas',good), ('Mal clasificados',testingSize-good), ('Precisión:',(good/testingSize)*100)])
  return dict_score['Precisión:'] #dict_score

### Scores

<div style="text-align: justify">Se desarrolló una función que toma como valores de entrada la base de datos dividida en atributos y clases, junto al tipo de discretización deseada. 
La función ejecuta los tres clasificadores con los datos ingresados $30$ veces y obtiene como salida la presición y desviación estándar calculadas. Para la discretización usando la librería de Sklearn se fijó el parámetro n_bins=10, sin embargo el valor se ajusta dependiendo de la base de datos ingresada. </div> 

In [9]:
def scores(X,y, kind):
    resultados={}
    if kind=="NP":
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="CAIM":
        X=main_caim(X,y)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="kmeans":
        enc = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="kmeans")
        X = enc.fit_transform(X)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="quantile":
        enc = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")
        X = enc.fit_transform(X)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}    
    return pd.DataFrame(resultados).T.describe().iloc[1:3].T

## Pre-processing data 

In [10]:
xlist=[]
ylist=[]

### Dataset 1 Iris

In [11]:
iris_database = load_iris()
X = iris_database.data
y = iris_database.target
X_discret = main_caim(X,y)
xlist.append(X)
ylist.append(y)

### Dataset 2 Skin

In [12]:
ds = pd.read_csv('Skin_NonSkin.csv', sep='\t', header=None)
ds = ds.sample(frac=0.9)
X = ds.iloc[:100,:-1]
X = np.array(X)
y = np.array(ds.iloc[:100,-1])
xlist.append(X)
ylist.append(y)

### Dataset 3 Cancer

In [13]:
cancer_dt = pd.read_csv('cancer.csv', header=None)
cancer_dt.dropna(how='all')
cancer_dt = cancer_dt.sample(frac=0.5)
x_dt_cancer = cancer_dt.iloc[:,:-2]
x_cancer = np.array(x_dt_cancer)

for i in range(x_cancer.shape[0]):
  for j in range(x_cancer.shape[1]):
    if (x_cancer[i][j] == '?'):
      x_cancer[i][j] = 0

X = np.array(x_cancer, dtype=float)
y = np.array(cancer_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 4 Wine

In [14]:
wine_dt = pd.read_csv('wine.csv', header=None)
x_dt_wine = wine_dt.iloc[:,:-1]
X = np.array(x_dt_wine)
y = np.array(wine_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 5 Vertebral





In [15]:
vertebral_dt = pd.read_csv('vertebral.csv', header=None)
x_dt_vertebral = vertebral_dt.iloc[:,:-1]
X = np.array(x_dt_vertebral)
y = np.array(vertebral_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 6 Sonar

In [16]:
sonar_dt = pd.read_csv('sonar.csv', header=None)
x_dt_sonar = sonar_dt.iloc[:,:-1]
X = np.array(x_dt_sonar)
y = np.array(sonar_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 7 Occupancy detection

In [17]:
segmentation_dt = pd.read_csv('occupancy_detection.csv', header=None)
segmentation_dt = segmentation_dt.sample(frac=0.1)
x_dt_segmentation = segmentation_dt.iloc[:,:-1]
X = np.array(x_dt_segmentation)
y = np.array(segmentation_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 8 Banknote authentication

In [18]:
segmentation_dt = pd.read_csv('data_banknote_authentication.csv', header=None)
x_dt_segmentation = segmentation_dt.iloc[:,:-1]
X = np.array(x_dt_segmentation)
y = np.array(segmentation_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 9 Glass identification

In [19]:
segmentation_dt = pd.read_csv('glass_identification.csv', header=None)
x_dt_segmentation = segmentation_dt.iloc[:,:-1]
X = np.array(x_dt_segmentation)
y = np.array(segmentation_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

### Dataset 10 Abalon

In [20]:
segmentation_dt = pd.read_csv('abalone.csv', header=None)
segmentation_dt = segmentation_dt.sample(frac=0.2)
x_dt_segmentation = segmentation_dt.iloc[:,:-1]
X = np.array(x_dt_segmentation)
y = np.array(segmentation_dt.iloc[:,-1])
xlist.append(X)
ylist.append(y)

## Execution

La función scores se aplicó iterativamente sobre cada base de datos para obtener el conjunto de métricas obtenidas a partir de todas las combinaciones posibles entre algoritmos de discretización y clasificación.

In [ ]:
#Recorriendo todos los dataset
#labels=["Dataset" + str(x) for x in range(10)]
labels = ["Iris", "Skin", "Cancer", "Wine", "Vertebral", "Sonar", "Occupancy", "Banknote", "Glass", "Abalon"]
kinds = ["NP", "CAIM", "kmeans", "quantile"]
allds=[]
for i in range(10):
    kindscores=[]
    for element in kinds:
        kindscores.append(scores(xlist[i],ylist[i],element))
        df = pd.concat(kindscores, axis = 1, keys=kinds).T
    allds.append(df)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


In [ ]:
df = pd.concat(allds, axis = 0, keys=labels)

In [ ]:
df.round(2)

#### Función auxiliar para plots

In [ ]:
def scores2(X,y, kind):
    resultados={}
    if kind=="NP":
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="CAIM":
        X=main_caim(X,y)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="kmeans":
        enc = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="kmeans")
        X = enc.fit_transform(X)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}
    elif kind=="quantile":
        enc = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")
        X = enc.fit_transform(X)
        for i in range(30):
            r1=naive_ingenuo(X,y,0.3)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            clf = DecisionTreeClassifier().fit(X_train, y_train)
            r2=cross_val_score(clf, X_test, y_test, cv=2)[0]
            clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
            r3=clf.score(X_test, y_test)
            resultados[i]={"NB":r1,"Arbol-Desicion":r2*100,"SVM":r3*100}    
    return pd.DataFrame(resultados)

## Visualizaciones

## Sin procesado de datos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,8)
rows = 2
cols = 5
axes=[]
fig=plt.figure()
for a in range(rows*cols):
    #b=imgs[a]
    axes.append( fig.add_subplot(rows, cols, a+1) )
    if a==10:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])
    else:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])

    axes[-1].set_title(subplot_title)  
    scores2(xlist[a],ylist[a],"NP").T.boxplot()
img=fig.tight_layout()    
plt.show(img)

## Aplicando CAIM

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,8)
rows = 2
cols = 5
axes=[]
fig=plt.figure()
for a in range(rows*cols):
    #b=imgs[a]
    axes.append( fig.add_subplot(rows, cols, a+1) )
    if a==10:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])
    else:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])

    axes[-1].set_title(subplot_title)  
    scores2(xlist[a],ylist[a],"CAIM").T.boxplot()
img=fig.tight_layout()    
plt.show(img)

## K-medias

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,8)
rows = 2
cols = 5
axes=[]
fig=plt.figure()
for a in range(rows*cols):
    #b=imgs[a]
    axes.append( fig.add_subplot(rows, cols, a+1) )
    if a==10:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])
    else:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])

    axes[-1].set_title(subplot_title)  
    scores2(xlist[a],ylist[a],"kmeans").T.boxplot()
img=fig.tight_layout()    
plt.show(img)

## Usando cuantiles

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,8)
rows = 2
cols = 5
axes=[]
fig=plt.figure()
for a in range(rows*cols):
    #b=imgs[a]
    axes.append( fig.add_subplot(rows, cols, a+1) )
    if a==10:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])
    else:
        subplot_title=("Presición por modelo para el DS:"+ labels[a])

    axes[-1].set_title(subplot_title)  
    scores2(xlist[a],ylist[a],"quantile").T.boxplot()
img=fig.tight_layout()    
plt.show(img)

## References

[1] Kurgan, L. A., & Cios, K. J. (2004). CAIM discretization algorithm. IEEE transactions on Knowledge and Data Engineering, 16(2), 145-153.

[2] [Scikit-learn: Machine Learning](https://scikit-learn.org/stable/index.html) in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

[3] Mitchell, T., Machine Learning. 1997.

[4] https://archive.ics.uci.edu/ml/index.php